In [1]:
import numpy as np
import qmt
import imt
import matplotlib.pyplot as plt
import pandas as pd
import quaternion_angle as qa
from pathlib import Path
import os

In [2]:
# LOAD DATA
path_data = "../../Daten/"
folder_data_imu = path_data + "Dict_Frames/"
folder_data_cam = path_data + "Knee_angle2d/"
filenames_imu = [str(f) for f in Path(folder_data_imu).rglob('*') if f.is_file()]  # all filnames in array
filenames_cam = [str(f) for f in Path(folder_data_cam).rglob('*') if f.is_file()]
sensors = ["S0333", "S1094", "S0593", "S0994", "S0477"] 
sequences = pd.read_csv(path_data + "sequences.txt", delimiter="\t", header=None, index_col=0)
# 1 und 41 ohne Sternum,25 rot, 26 S0194 fehlt, 53 keine Kamera datei, 35 S0994 fehlt, 18 fehlt, ab 54]
seq_cam_start = sequences.iloc[:, 0:1].values.ravel()  # CAM start frame
seq_imu_start = sequences.iloc[:, 1:2].values.ravel()   # IMU start frame
seq_names = sequences.index.values              # Sequenz Namen
Hz_imu = 52
Hz_cam = 30 # Kamera sample Rate
Hz_ring = 100

In [ ]:
qhat_all = []
extras_all = []
#angle_kr_all = []
#angle_kl_all =
qa_kr_all = []
qa_kl_all = []
ts_ring_all = []

for i_seq in range(len(filenames_imu)):
    # LOAD DATA
    data_imu_seq = np.load(filenames_imu[i_seq], allow_pickle=True).item()

    # PREPARE DATA
    imu_data = {
        i: dict(acc=data_imu_seq[sensors[i]]["acc"][seq_imu_start[i_seq]:], gyr=data_imu_seq[sensors[i]]["gyr_rad"][seq_imu_start[i_seq]:])
        for i in range(len(sensors))  
        }
    imu_data = imt.utils.resample(imt.utils.crop_tail(imu_data, Hz_imu), Hz_imu, Hz_ring)

    imu_data[0] = dict(
        acc=qmt.rotate(qmt.quatFromAngleAxis(-np.pi, [0, 0, 1]), imu_data[0]["acc"]),
        gyr=qmt.rotate(qmt.quatFromAngleAxis(-np.pi, [0, 0, 1]), imu_data[0]["gyr"]),
    )
    
    # ESTIMATE ORIENTATIONS
    rel_method = imt.methods.RING(axes_directions=np.array([1.0, 0, 0]))
    graph = [-1, 0, 1, 0, 3]
    solver = imt.Solver(graph, [imt.methods.VQF(offline=True)] +
                        #[imt.wrappers.JointTracker1D(rel_method)]*4, 
                        [rel_method]*4, 
                        Ts=1/Hz_ring)
    qhat, extras = solver.step(imu_data)
    qhat_all.append(qhat)
    extras_all.append(extras)

    q1 = qhat[1]
    q2 = qhat[2] 
    q3 = qhat[3] 
    q4 = qhat[4] 
    # Compute angles
    qa_kr = qa.quaternion_angle(q1, q2)
    qa_kl = qa.quaternion_angle(q3, q4)  

    # Extract timesteps
    T = qhat[0].shape[0]
    ts = np.round(np.arange(T)/Hz_ring, 2)

    # Extract measruement values if JointTracker1d is used
    #angle_kr = -np.rad2deg(extras[2]["joint_angle_rad"])
    #angle_kl = -np.rad2deg(extras[4]["joint_angle_rad"]) 

    #angle_kr_all.append(angle_kr)
    #angle_kl_all.append(angle_kl)
    qa_kr_all.append(qa_kr)
    qa_kl_all.append(qa_kl)
    ts_ring_all.append(ts)

`crop_tail`: Crop off at t=177.15384615384616s
`crop_tail`: Crop off at t=181.15384615384616s
`crop_tail`: Crop off at t=180.6153846153846s
`crop_tail`: Crop off at t=199.98076923076923s
`crop_tail`: Crop off at t=124.61538461538461s
`crop_tail`: Crop off at t=219.07692307692307s
`crop_tail`: Crop off at t=184.0s
`crop_tail`: Crop off at t=183.30769230769232s
`crop_tail`: Crop off at t=199.84615384615384s
`crop_tail`: Crop off at t=179.69230769230768s
`crop_tail`: Crop off at t=185.92307692307693s
`crop_tail`: Crop off at t=180.07692307692307s
`crop_tail`: Crop off at t=185.46153846153845s
`crop_tail`: Crop off at t=110.0s
`crop_tail`: Crop off at t=181.92307692307693s
`crop_tail`: Crop off at t=182.3846153846154s
`crop_tail`: Crop off at t=205.53846153846155s
`crop_tail`: Crop off at t=184.67307692307693s
`crop_tail`: Crop off at t=180.30769230769232s
`crop_tail`: Crop off at t=179.53846153846155s
`crop_tail`: Crop off at t=192.23076923076923s
`crop_tail`: Crop off at t=181.4615384615

In [ ]:
# CAM DATA
cam_kr_all = []
cam_kl_all = []
ts_cam_all = []

for i_seq in range(len(filenames_imu)):
    data_cam_both_seq = np.loadtxt(filenames_cam[i_seq], delimiter=",") # Kniewinkel aus Videodaten beide Beine
    data_cam_kr_seq = data_cam_both_seq[:,1][seq_cam_start[i_seq]:]      # [:,1] nur Knie rechts, [:,0] nur Knie links
    data_cam_kl_seq = data_cam_both_seq[:,0][seq_cam_start[i_seq]:]      # [i:] alles vor index i entfernen

    ts_cam = np.round(np.arange(len(data_cam_kr_seq))*(1/Hz_cam), 2) # timesteps for camera data with camera sample rate
    cam_kr_all.append(data_cam_kr_seq)
    cam_kl_all.append(data_cam_kl_seq)
    ts_cam_all.append(ts_cam)


NameError: name 'filenames_imu' is not defined

In [ ]:
# RAW IMU DATA
imu_data_all = []
ts_imu_all =[]

for i_seq in range(len(filenames_imu)):
    # LOAD DATA
    data_imu_seq = np.load(filenames_imu[i_seq], allow_pickle=True).item()

    # PREPARE DATA
    imu_data = {
        i: dict(acc=data_imu_seq[sensors[i]]["acc"][seq_imu_start[i_seq]:], gyr=data_imu_seq[sensors[i]]["gyr_rad"][seq_imu_start[i_seq]:])
        for i in range(len(sensors))
        }
    imu_data = imt.utils.crop_tail(imu_data, 52)

    imu_data[0] = dict(
        acc=qmt.rotate(qmt.quatFromAngleAxis(-np.pi, [0, 0, 1]), imu_data[0]["acc"]),
        gyr=qmt.rotate(qmt.quatFromAngleAxis(-np.pi, [0, 0, 1]), imu_data[0]["gyr"]),
    )

    ts_imu = np.round(np.arange(len(imu_data[0]["acc"]))*(1/Hz_imu), 2) # timesteps for imu data with imu sample rate

    imu_data_all.append(imu_data)
    ts_imu_all.append(ts_imu)

# Prepare dataset for machine learning algorithms by dewrapping arrays
S0333_acc = np.vstack([entry[0]['acc'] for entry in imu_data_all])
S0333_gyr = np.vstack([entry[0]['gyr'] for entry in imu_data_all])
S1094_acc = np.vstack([entry[1]['acc'] for entry in imu_data_all])
S1094_gyr = np.vstack([entry[1]['gyr'] for entry in imu_data_all])
S0593_acc = np.vstack([entry[2]['acc'] for entry in imu_data_all])
S0593_gyr = np.vstack([entry[2]['gyr'] for entry in imu_data_all])
S0994_acc = np.vstack([entry[3]['acc'] for entry in imu_data_all])
S0994_gyr = np.vstack([entry[3]['gyr'] for entry in imu_data_all])
S0477_acc = np.vstack([entry[4]['acc'] for entry in imu_data_all])
S0477_gyr = np.vstack([entry[4]['gyr'] for entry in imu_data_all])
imu_data_ml = np.hstack((S0333_acc, S0333_gyr, S1094_acc, S1094_gyr, S0593_acc, S0593_gyr, S0994_acc, S0994_gyr, S0477_acc, S0477_gyr))

In [ ]:
#### RESAMPLING DATA to IMU FREQUENCY
qa_kr_resample_all = []
qa_kl_resample_all = []
cam_kr_resample_all = []
cam_kl_resample_all = []
ts_ring_resample_all = []
ts_cam_resample_all = []

for i in range(len(filenames_imu)):
    factor_ring = Hz_ring/Hz_imu
    factor_cam = Hz_cam/Hz_imu

    indices_ring = np.round(np.arange(0, len(ts_ring_all[i]), factor_ring), 1)  # Select every nth index
    indices_cam = np.round(np.arange(0, len(ts_cam_all[i]), factor_ring), 1)  # Select every nth index

    ts_ring_resample = np.round(np.interp(indices_ring, np.arange(len(ts_ring_all[i])), ts_ring_all[i]), 2)
    ts_cam_resample = np.round(np.interp(indices_cam, np.arange(len(ts_cam_all[i])), ts_ring_all[i]), 2)
    #angle_kr_resample = np.interp(indices, np.arange(len(ts)), angle_kr)
    #angle_kl_resample = np.interp(indices, np.arange(len(ts)), angle_kl)
    qa_kr_resample = np.interp(indices_ring, np.arange(len(ts_ring_all[i])), qa_kr_all[i])  
    qa_kl_resample = np.interp(indices_ring, np.arange(len(ts_ring_all[i])), qa_kl_all[i]) 
    cam_kr_resample = np.interp(indices_cam, np.arange(len(ts_cam_all[i])), cam_kr_all[i])  
    cam_kl_resample = np.interp(indices_cam, np.arange(len(ts_cam_all[i])), cam_kl_all[i]) 

    qa_kr_resample_all.append(qa_kr_resample)
    qa_kl_resample_all.append(qa_kl_resample)
    cam_kr_resample_all.append(cam_kr_resample)
    cam_kl_resample_all.append(cam_kl_resample)
    ts_ring_resample_all.append(ts_ring_resample)
    ts_cam_resample_all.append(ts_cam_resample)

In [ ]:

ring_angles_kl_52Hz_i = imt.utils.resample(ring_angles_kl[i], Hz_cam, Hz)
ring_angles_kl_52Hz.append(ring_angles_kl_52Hz_i)
#ring_angles_kr_52Hz = imt.utils.resample(imt.utils.crop_tail(ring_angles_kr, Hz_cam), Hz_cam, Hz)
#cam_angles_kl_52Hz = imt.utils.resample(imt.utils.crop_tail(cam_angles_kl, Hz_cam), Hz_cam, Hz)
##cam_angles_kr_52Hz = imt.utils.resample(imt.utils.crop_tail(cam_angles_kr, Hz_cam), Hz_cam, Hz)

In [ ]:
# SAVE RESULTS RING
path_results = "./results" 
os.makedirs(path_results, exist_ok=True) # check if directory exists
np.save(os.path.join(path_results, "Angles_IMU_RING_Knee_Left.npy"), np.array(qa_kl_all, dtype=object))  # Save files to results path
np.save(os.path.join(path_results, "Angles_IMU_RING_Knee_Right.npy"), np.array(qa_kr_all, dtype=object))
np.save(os.path.join(path_results, "Timesteps_RING.npy"), np.array(ts_ring_all, dtype=object))
np.save(os.path.join(path_results, "Angles_IMU_RING_Knee_Left_52.npy"), np.array(qa_kl_resample_all, dtype=object))  # Save files to results path
np.save(os.path.join(path_results, "Angles_IMU_RING_Knee_Right_52.npy"), np.array(qa_kr_resample_all, dtype=object))
np.save(os.path.join(path_results, "Timesteps_RING_52.npy"), np.array(ts_ring_resample_all, dtype=object))
np.save(os.path.join(path_results, "Sequences_names.npy"), np.array(seq_names, dtype=object))

Saved successfully to: ./results


In [ ]:
# SAVE RESULTS CAM
np.save(os.path.join(path_results, "Angles_CAM_Knee_Left.npy"), np.array(cam_kl_all, dtype=object))
np.save(os.path.join(path_results, "Angles_CAM_Knee_Right.npy"), np.array(cam_kr_all, dtype=object))
np.save(os.path.join(path_results, "Timesteps_CAM.npy"), np.array(ts_cam_all, dtype=object))
np.save(os.path.join(path_results, "Angles_CAM_Knee_Left_52.npy"), np.array(cam_kl_resample_all, dtype=object))
np.save(os.path.join(path_results, "Angles_CAM_Knee_Right_52.npy"), np.array(cam_kr_resample_all, dtype=object))
np.save(os.path.join(path_results, "Timesteps_CAM_52.npy"), np.array(ts_cam_resample_all, dtype=object))

Saved successfully to: ./results


In [ ]:
# SAVE RESULTS RAW IMU
np.save(os.path.join(path_results, "IMU_data_ml.npy"), np.array(imu_data_ml, dtype=object))
np.save(os.path.join(path_results, "IMU_data_all.npy"), np.array(imu_data_all, dtype=object))

In [ ]:
rmse_value3 = np.sqrt(np.mean((qa_kr_all[1] - cam_kr_all[1][:len(qa_kr_all[1])]) ** 2))
print(f"RMSE: {rmse_value3:.4f}")

RMSE: 18.9581
